In [1]:
import os

def get_file_pairs(exchanges):
    filenames = []
    for directory in os.listdir('ohlcv_data'):
        if directory != '.DS_Store':
            for filename in os.listdir('ohlcv_data/' + directory):
                if filename.endswith('300.csv'):
                     filenames.append(filename)
    file_pairs = []
    for filename_1 in filenames:
        remaining_filenames = filenames[filenames.index(filename_1)+1:]
        for filename_2 in remaining_filenames:
            for exchange in exchanges:
                if filename_1.replace(exchange, '') in filename_2:
                    file_pairs.append([filename_1, filename_2])
    return file_pairs

exchanges = ['bitfinex', 'coinbase_pro', 'hitbtc']
get_file_pairs(exchanges)

[['bitfinex_eos_usdt_300.csv', 'hitbtc_eos_usdt_300.csv'],
 ['bitfinex_bch_btc_300.csv', 'coinbase_pro_bch_btc_300.csv'],
 ['bitfinex_bch_btc_300.csv', 'hitbtc_bch_btc_300.csv'],
 ['bitfinex_etc_usd_300.csv', 'coinbase_pro_etc_usd_300.csv'],
 ['bitfinex_btc_usd_300.csv', 'coinbase_pro_btc_usd_300.csv'],
 ['bitfinex_ltc_btc_300.csv', 'coinbase_pro_ltc_btc_300.csv'],
 ['bitfinex_ltc_btc_300.csv', 'hitbtc_ltc_btc_300.csv'],
 ['bitfinex_dash_usd_300.csv', 'coinbase_pro_dash_usd_300.csv'],
 ['bitfinex_dash_btc_300.csv', 'coinbase_pro_dash_btc_300.csv'],
 ['bitfinex_dash_btc_300.csv', 'hitbtc_dash_btc_300.csv'],
 ['bitfinex_ltc_usd_300.csv', 'coinbase_pro_ltc_usd_300.csv'],
 ['bitfinex_bch_usdt_300.csv', 'hitbtc_bch_usdt_300.csv'],
 ['bitfinex_bch_usd_300.csv', 'coinbase_pro_bch_usd_300.csv'],
 ['bitfinex_eos_usd_300.csv', 'coinbase_pro_eos_usd_300.csv'],
 ['bitfinex_xrp_usd_300.csv', 'coinbase_pro_xrp_usd_300.csv'],
 ['bitfinex_eth_btc_300.csv', 'coinbase_pro_eth_btc_300.csv'],
 ['bitfinex_

In [2]:
import pandas as pd

def get_df(filename):
    df = pd.read_csv(filename, index_col=0)
    return df

In [3]:
def resample_ohlcv(df, period='5T'):

    # Set date as the index. This is needed for the function to run
    df = df.set_index(['date'])

    # Aggregation function
    ohlc_dict = {                                                                                                             
    'open':'first',                                                                                                    
    'high':'max',                                                                                                       
    'low':'min',                                                                                                        
    'close': 'last',                                                                                                    
    'base_volume': 'sum'
    }

    # Apply resampling.
    df = df.resample(period, how=ohlc_dict, closed='left', label='left')
    
    return df

In [4]:
from ta import add_all_ta_features

def fill_nan(df):
    
    df['close'] = df['close'].ffill()
    df = df.bfill(axis=1)

    return df

def engineer_features(df):
    
    df['date'] = pd.to_datetime(df['closing_time'], unit='s')
    df = resample_ohlcv(df)
    df = df.reset_index()
    
    df['date'] = df['date'].astype('int64')//1e9
    df = df.rename(columns={'date': 'closing_time'})
        
    df['nan_ohlcv'] = df['close'].apply(lambda x: 1 if pd.isnull(x) else 0)
    df = fill_nan(df)
        
    df = add_all_ta_features(df, 'open', 'high', 'low', 'close',
                             'base_volume', fillna=True)
    
    df['closing_time'] = df['closing_time'].astype('int64')
    df['nan_ohlcv'] = df['nan_ohlcv'].astype('int64')
    
    df = df.drop(columns=['open', 'high', 'low', 'momentum_kama',
                          'momentum_stoch', 'others_cr', 'others_dlr',
                          'trend_ema_fast', 'trend_ema_slow', 
                          'trend_ichimoku_a', 'trend_ichimoku_b', 'trend_kst',
                          'trend_macd', 'trend_visual_ichimoku_a',
                          'trend_visual_ichimoku_b', 'volatility_bbh',
                          'volatility_bbl', 'volatility_bbm',
                          'volatility_dch', 'volatility_dcl',
                          'volatility_kcc', 'volatility_kch',
                          'volatility_kcl'])
       
    return df

In [5]:
def get_higher_closing_price(df):
    if (df['close_exchange_1'] - df['close_exchange_2']) > 0:
        return 'exchange_1'
    elif (df['close_exchange_1'] - df['close_exchange_2']) < 0:
        return 'exchange_2'
    else:
        return 'equivalent'

def get_pct_higher(df):
    if df['higher_closing_price'] == 'exchange_1':
        return ((df['close_exchange_1'] / 
                 df['close_exchange_2'])-1)*100
    elif df['higher_closing_price'] == 'exchange_2':
        return ((df['close_exchange_2'] / 
                 df['close_exchange_1'])-1)*100
    else:
        return 0
    
def get_arbitrage_opportunity(df):
    if df['pct_higher'] < .55:
        return 0 # no arbitrage
    elif df['higher_closing_price'] == 'exchange_1':
        return -1 # arbitrage exchange 2 to exchange 1
    elif df['higher_closing_price'] == 'exchange_2':
        return 1 # arbitrage exchange 1 to exchange 2
    
def get_window_length(df):
    target_list = df['arbitrage_opportunity'].to_list()
    window_length = 5
    window_lengths = []
    for i in range(len(target_list)):
            if target_list[i] == target_list[i-1]:
                window_length += 5
                window_lengths.append(window_length)
            else:
                window_length = 5
                window_lengths.append(window_length)      
    df['window_length'] = window_lengths
    return df
        
def merge_dfs(df1, df2):
    df = pd.merge(df1, df2, on='closing_time',
                  suffixes=('_exchange_1', '_exchange_2'))
        
    df['year'] = pd.to_datetime(df['closing_time'], unit='s').dt.year
    df['month'] = pd.to_datetime(df['closing_time'], unit='s').dt.month
    df['day'] = pd.to_datetime(df['closing_time'], unit='s').dt.day

    df['higher_closing_price'] = df.apply(get_higher_closing_price, axis=1)
    df['pct_higher'] = df.apply(get_pct_higher, axis=1)
    df['arbitrage_opportunity'] = df.apply(get_arbitrage_opportunity, axis=1)
    df = get_window_length(df)
    df = df.drop(columns=['higher_closing_price', 'pct_higher'])
    df = df[:-8]
    return df

In [6]:
interval=30

def get_target_value(df, interval=interval):

    if df['window_length_shift'] >= interval:
        if df['arbitrage_opportunity_shift'] == 1:
            return 1
        elif df['arbitrage_opportunity_shift'] == -1:
            return -1
        elif df['arbitrage_opportunity_shift'] == 0:
            return 0
    else:
        return 0
    
def get_target(df, interval=interval):
    
    rows_to_shift = int(-1*(interval/5))
    
    df['arbitrage_opportunity_shift'] = df['arbitrage_opportunity'].shift(
        rows_to_shift)
    df['window_length_shift'] = df['window_length'].shift(rows_to_shift)
    
    df['target'] = df.apply(get_target_value, axis=1)
    
    df = df.drop(columns=['window_length_shift',
                          'arbitrage_opportunity_shift'])
    
    return df

In [7]:
def get_higher_closing_price(df):
    if (df['close_exchange_1'] - df['close_exchange_2']) > 0:
        return 'exchange_1'
    elif (df['close_exchange_1'] - df['close_exchange_2']) < 0:
        return 'exchange_2'
    else:
        return 'equivalent'
    
def get_close_shift(df, interval=interval):
    rows_to_shift = int(-1*(interval/5))
    df['close_exchange_1_shift'] = df['close_exchange_1'].shift(rows_to_shift)
    df['close_exchange_2_shift'] = df['close_exchange_2'].shift(rows_to_shift)
    return df

def get_profit(df):
    if df['higher_closing_price'] == 'exchange_1':
        return (((df['close_exchange_1_shift'] / 
                 df['close_exchange_2'])-1)*100)-.55
    elif df['higher_closing_price'] == 'exchange_2':
        return (((df['close_exchange_2_shift'] / 
                 df['close_exchange_1'])-1)*100)-.55
    else:
        return 0

In [ ]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

def create_all_arbitrage_dfs_and_models(exchanges):
    for pair in get_file_pairs(exchanges):
        for exchange in exchanges:
            if exchange in pair[0]:
                exchange_1 = exchange
            if exchange in pair[1]:
                exchange_2 = exchange
        
        df1 = get_df('ohlcv_data/' + exchange_1 + '_300/' + pair[0])
        print('engineering df1...')
        df1 = engineer_features(df1)
        print('success!')

        df2 = get_df('ohlcv_data/' + exchange_2 + '_300/' + pair[1])
        print('engineering df2...')
        df2 = engineer_features(df2)
        print('success!')

        print('merging df1 and df2...')
        df = merge_dfs(df1, df2)
        print('success!')
        
        end_of_model_name = '_' + pair[1].replace('_300.csv', '.pkl')
        model_name = exchange_1 + end_of_model_name
        print(model_name.replace('.pkl', '').upper())
        
        df = get_target(df)
        
        test_train_split_row = round(len(df)*.7)
        test_train_split_time = df['closing_time'][test_train_split_row]

        train_cutoff_time = test_train_split_time - 604800
        test_cutoff_time = test_train_split_time + 604800

        train = df[df['closing_time'] < train_cutoff_time]
        test = df[df['closing_time'] > test_cutoff_time]
        print('train and test shape:'.format(model=model_name), 
              train.shape, test.shape)

        features = df.drop(columns=['target']).columns.tolist()
        target = 'target'

        X_train = train[features]
        X_test = test[features]
        y_train = train[target]
        y_test = test[target]
        
        model = RandomForestClassifier(max_depth=75, n_estimators=100, 
                                       n_jobs=-1, random_state=42)
        
        if X_train.shape[0] > 1000:
            model.fit(X_train, y_train)
            print('model fitted!')

            train_score = model.score(X_train, y_train)
            print('train accuracy:', train_score)

            y_preds = model.predict(X_test)
            print('predictions made!')

            score = accuracy_score(y_test, y_preds)
            print('test accuracy:', score)

            pickle.dump(model, open('pickles/{model}.pkl'.format(
                model=model_name), 'wb'))
            print('pickle saved!'.format(model=model) + '\n')
                
            unique_y_test = y_test.unique().tolist()
            unique_y_preds = list(set(y_preds))
            labels = list(set(unique_y_test + unique_y_preds))
            labels.sort()
            columns = [f'Predicted {label}' for label in labels]
            index = [f'Actual {label}'  for label in labels]
                
            confusion = pd.DataFrame(confusion_matrix(y_test, y_preds),
                                     columns=columns, index=index)
                
            print(model_name + ' confusion matrix:')
            print(confusion, '\n')
                
            test_with_preds = X_test
            test_with_preds['higher_closing_price'
                           ] = test_with_preds.apply(
                get_higher_closing_price, axis=1)
            test_with_preds = get_close_shift(test_with_preds)
            test_with_preds['pred'] = y_preds
            test_with_preds['pct_profit'] = test_with_preds.apply(
                get_profit, axis=1).shift(-1)
                
            pct_profit_mean = test_with_preds[test_with_preds[
                'pred'] != 0]['pct_profit'].mean()
            pct_profit_median = test_with_preds[test_with_preds[
                'pred'] != 0]['pct_profit'].median()
                
            print('percent profit mean:', pct_profit_mean)
            print('percent profit median:', pct_profit_median, '\n\n')

        else:
            print('not enough data!'.format(model=model_name))

exchanges = ['bitfinex', 'coinbase_pro', 'hitbtc']
create_all_arbitrage_dfs_and_models(exchanges)

engineering df1...


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  app.launch_new_instance()
/anaconda3/lib/python3.7/site-packages/ta/trend.py:170: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
/anaconda3/lib/python3.7/site-packages/ta/trend.py:174: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])
